In [ ]:
#feature_extractor.py

import os
import pickle

actors = os.listdir('/content/drive/MyDrive/celedata')

filenames = []

for actor in actors:
    for file in os.listdir(os.path.join('/content/drive/MyDrive/celedata',actor)):
        filenames.append(os.path.join('/content/drive/MyDrive/celedata',actor,file))

pickle.dump(filenames,open('/content/drive/MyDrive/filenames.pkl','wb'))

from tensorflow.keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import numpy as np
import pickle
from tqdm import tqdm

filenames = pickle.load(open('/content/drive/MyDrive/filenames.pkl','rb'))

model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

def feature_extractor(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img = np.expand_dims(img_array,axis=0)
    preprocessed_img = preprocess_input(expanded_img)

    result = model.predict(preprocessed_img).flatten()

    return result

features = []

for file in tqdm(filenames):
    features.append(feature_extractor(file,model))

pickle.dump(features,open('/content/drive/MyDrive/embedding.pkl','wb'))




In [ ]:
#streamlit_app.py

import streamlit as st 
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
from PIL import Image

import os
import cv2
from mtcnn import MTCNN
import numpy as np

detector = MTCNN()
model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')
feature_list = pickle.load(open('/content/drive/MyDrive/embedding.pkl','rb'))
filenames = pickle.load(open('/content/drive/MyDrive/filenames.pkl','rb'))

def save_uploaded_image(uploaded_image):
    try:
        with open(os.path.join('uploads',uploaded_image.name),'wb') as f:
            f.write(uploaded_image.getbuffer())
        return True
    except:
        return False

def extract_features(img_path,model,detector):
    img = cv2.imread(img_path)
    results = detector.detect_faces(img)

    x, y, width, height = results[0]['box']

    face = img[y:y + height, x:x + width]

    #  extract its features
    image = Image.fromarray(face)
    image = image.resize((224, 224))

    face_array = np.asarray(image)

    face_array = face_array.astype('float32')

    expanded_img = np.expand_dims(face_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img)
    result = model.predict(preprocessed_img).flatten()
    return result

def recommend(feature_list,features):
    similarity = []
    for i in range(len(feature_list)):
        similarity.append(cosine_similarity(features.reshape(1, -1), feature_list[i].reshape(1, -1))[0][0])

    index_pos = sorted(list(enumerate(similarity)), reverse=True, key=lambda x: x[1])[0][0]
    return index_pos

st.title('Which bollywood celebrity are you?')

uploaded_image = st.file_uploader('Choose an image')

if uploaded_image is not None:
    # save the image in a directory
    if save_uploaded_image(uploaded_image):
        # load the image
        display_image = Image.open(uploaded_image)

        # extract the features
        features = extract_features(os.path.join('uploads',uploaded_image.name),model,detector)
        # recommend
        index_pos = recommend(feature_list,features)
        predicted_actor = " ".join(filenames[index_pos].split('/')[-2].split('_'))
        # display
        st.header("You Look Like: "+predicted_actor)
        col1,col2 = st.columns(2)

        with col1:
            st.image(display_image,width=250)
            st.header("Your Image")
        with col2:
            st.image(filenames[index_pos],width=250)
            st.header("Predicted Celebrity")


In [ ]:
!streamlit run /content/streamlit_app.py & npx localtunnel --port 8501

[############......] - finalize:yargs: sill finalize /root/.npm/_npx/1449/lib/n

npx: installed 22 in 3.606s
your url is: https://pink-shrimps-cheat-104-154-241-46.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.154.241.46:8501

2022-11-28 06:03:08.894774: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
1/1 [==============================] - 0s 208ms/step
2022-11-28 06:15:21.564 5 out of the last 14 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fe961504950> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has redu